# PAWPULARITY PETFINDER

![Photo by Eddie Galaxy from Pexels](https://images.pexels.com/photos/3628100/pexels-photo-3628100.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)

> Saving one animal won't change the world, but it will change the world for that one animal.

## Objective
**Predict engagement with a pet's profile based on the photograph for that profile**

To predict engagement we have to predict pawpularity score, which is derived from each pet profile's page view statistics at the listing pages, using an algorithm that normalizes the traffic data across different pages, platforms (web & mobile) and various metrics.

## Evaluation
**RMSE (Root Mean Square Error)**

$$ \Large RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2} $$

## Internet Disabled
Here since we are using wandb and pretrained model we have to enable internet, for submission we will create a new notebook with intenet disabled. Here is an inference [notebook](https://www.kaggle.com/tarunbisht11/find-a-pet-with-lightning-speed-inference).

### What's New
- Training with GPU and TPU
- Saving OOF files for later blending
- Removing unnecessary code sections that are used for inference only, that will be part of inference notebook only
- model used : tf_efficientnet_b2 + meta
- added functionality to freeze backbone also through config
- if pretrained backbone is freeze we can use class information we get from model's classfier as extra data.
- make logger to pause when we are in debug mode for testing code.

# Configurations

In [ ]:
class Config:
    '''general'''
    debug = False
    seed = 42
    num_workers = 4
    use_tpu = False
    
    '''data'''
    batch_size = 64
    target_col = "Pawpularity"
    
    '''model'''
    # 'regnety_002', 'efficientnet_b3', 'efficientnet_b0', 
    # 'vit_base_patch16_224', 'tf_efficientnet_b4_ns'
    model_name = 'tf_efficientnet_b2'
    # number of predictors
    targets = 1
    # regressor features, number of useful features in meta data
    # 12 means using all columns except id and target column
    num_features = 12
    # input image size send to network can be like 256, 512, 768, 1028
    input_size = 260
    # freeze backbone network
    freeze_backbone = False
    
    '''training'''
    n_fold = 5
    trn_folds = [0, 1, 2, 3, 4]
    epochs = 1000
    print_freq = 10
    lr = 1e-4
    train = True
    # for fp16 training change it to 16
    precision = 16
    patience = 5
    
    '''gradients'''
    #adamw', 'adam'
    optimizer = 'adamw'
    weight_decay = 1e-6
    gradient_accumulation_steps= 1
    max_grad_norm = 1000
    
    '''lr scheduler'''
    #'ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts'
    lr_scheduler = 'CosineAnnealingWarmRestarts'
    factor = 0.2 # ReduceLROnPlateau
    patience = 4 # ReduceLROnPlateau
    eps = 1e-6 # ReduceLROnPlateau
    T_max = 15 # CosineAnnealingLR
    T_0 = 10 # CosineAnnealingWarmRestarts
    min_lr = 1e-6
    warmup_epochs = 0
    multiplier = 10
    
    '''loggers -> weights and bias'''
    loggers = True
    project = "pet-pawpularity"
    group = "gpu-training"

# Installing dependencies

### Installing
- tpu pytorch if using tpus
- timm vision models library

In [ ]:
import subprocess
if Config.use_tpu:
    if not subprocess.check_call("curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py", shell=True):
        print("setting up python xla")
    else:
        print("cannot setup pytorch xla")
    if not subprocess.check_call("python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev", shell=True):
        print("successfuly installed xla and its dependencies")
    else:
        print("cannot install xla and dependencies")

In [ ]:
! pip install -qq ../input/timm-pytorch-image-models/pytorch-image-models-master

In [ ]:
import os
import gc
import random
import time
import math

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import torchmetrics as metrics
import timm

from kaggle_secrets import UserSecretsClient
from kaggle_datasets import KaggleDatasets

import sklearn.model_selection as ms

import wandb

import warnings
warnings.filterwarnings('ignore')

### wandb login

In [ ]:
# before running add your api secret inside Add-ons > Secrets
os.environ["WANDB_SILENT"] = "true"
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")
wandb.login(key=secret_value_0)

# Loading Data

In [ ]:
train_csv = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_csv = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
train_data_path = "../input/petfinder-pawpularity-score/train"
test_data_path = "../input/petfinder-pawpularity-score/test"

In [ ]:
train_csv["path"] = train_csv["Id"].apply(lambda x: os.path.join(train_data_path, x+".jpg"))
test_csv["path"] = test_csv["Id"].apply(lambda x: os.path.join(test_data_path, x+".jpg"))

In [ ]:
train_csv.head()

In [ ]:
test_csv.head()

In [ ]:
# if debug mode use only 1000 samples for testing only make debug=False for actual training
if Config.debug:
    print("debug mode on")
    Config.epochs = 2
    Config.print_freq = 100
    Config.trn_folds = [0]
    Config.loggers = False
    #train_csv = train_csv.sample(n=1000, random_state=Config.seed).reset_index(drop=True)
else:
    Config.loggers = True
    print("debug mode off")

In [ ]:
pl.seed_everything(Config.seed)

# Basic EDA

In [ ]:
len(train_csv)

In [ ]:
train_csv.shape

In [ ]:
len(test_csv)

In [ ]:
test_csv.shape

In [ ]:
train_csv.columns

In [ ]:
train_csv.isnull().sum()

In [ ]:
test_csv.isnull().sum()

In [ ]:
for col in train_csv.columns:
    if col not in ["Id", Config.target_col, "path"]:
        print(train_csv[col].value_counts())

# Define Helpers

### Cross validation folds creation validation helper

In [ ]:
class CrossValidation:
    def __init__(self, df, shuffle,random_state=None):
        self.df = df
        self.random_state = random_state
        self.shuffle = shuffle
        if shuffle is True:
            self.df = df.sample(frac=1,
                random_state=self.random_state).reset_index(drop=True)
        else:
            self.random_state=None

    def hold_out_split(self,percent,stratify=None):
        if stratify is not None:
            y = self.df[stratify]
            train,val = ms.train_test_split(self.df, test_size=percent/100,
                stratify=y, random_state=self.random_state)
            return train,val
        size = len(self.df) - int(len(self.df)*(percent/100))
        train = self.df.iloc[:size,:]
        val = self.df.iloc[size:,:]
        return train, val

    def kfold_split(self, splits, stratify=None):
        if stratify is not None:
            kf = ms.StratifiedKFold(n_splits=splits, 
                                    shuffle=self.shuffle,
                                    random_state=self.random_state
                                    )
            y = self.df[stratify]
            for train, val in kf.split(X=self.df,y=y):
                t = self.df.iloc[train,:]
                v = self.df.iloc[val, :]
                yield t, v
        else:
            kf = ms.KFold(n_splits=splits, shuffle=self.shuffle,
                random_state=self.random_state)
            for train, val in kf.split(X=self.df):
                t = self.df.iloc[train,:]
                v = self.df.iloc[val, :]
                yield t, v

### Dataset loader helper

In [ ]:
class PawpularityDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, augmentations=None, test=False):
        super(PawpularityDataset, self).__init__()
        self.csv = csv_file
        self.test = test
        self.augs = augmentations
        self.length = len(self.csv)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        path = self.csv.iloc[idx]["path"]
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.augs is not None:
            img = self.augs(image=img)['image']
        else:
            img = torch.from_numpy(img).float()
            img = img.permute(2, 0, 1)
        meta = torch.from_numpy(self.csv.iloc[idx, 1:13].values.astype(np.float32))
        if self.test:
            return img, meta
        label = torch.tensor(self.csv.iloc[idx]["Pawpularity"], dtype=torch.float)
        return img, meta, label

### Data Augmentation helpers

In [ ]:
class ImageAugmentations:
    '''
        image_size: resize image to -> (width, height)
        train_augs: include augmentations like random crop, rotation etc training if false then return
                    only resize image as pytorch tensor
    '''
    def __init__(self, image_size, apply_augs=False):
        self.image_size = image_size
        self.apply_augs = apply_augs
        
    def train_augs(self):
        if self.apply_augs:
            return A.Compose([A.Resize(self.image_size, self.image_size),
                              A.HorizontalFlip(p=.5),
                              A.ChannelShuffle(p=.1),
                              A.ColorJitter(p=.2),
                              A.RandomGamma(p=.1),
                              A.Sharpen(p=.1),
                              A.Cutout(p=0.2),
                              # imagenet normalization
                              A.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],
                                          max_pixel_value=255.0,
                                          p=1.0),
                              ToTensorV2()])
        return A.Compose([A.Resize(self.image_size, self.image_size),
                          A.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225],
                                      max_pixel_value=255.0,
                                      p=1.0),
                          ToTensorV2()])
    
    def valid_augs(self):
        return A.Compose([A.Resize(self.image_size, self.image_size),
                          A.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225],
                                      max_pixel_value=255.0,
                                      p=1.0),
                          ToTensorV2()])
        

### plotting images from dataset

In [ ]:
augs = ImageAugmentations(Config.input_size, apply_augs=True)
ds = PawpularityDataset(train_csv, augmentations=augs.train_augs())
loader = torch.utils.data.DataLoader(ds,
                                     shuffle=True,
                                     batch_size=16)
images, meta, labels = next(iter(loader))
plt.figure(figsize=(15, 15))
for step, (image, label) in enumerate(zip(images, labels)):
    plt.subplot(4, 4, step+1)
    plt.imshow(image.permute(1, 2, 0).numpy())
    plt.axis('off')
    plt.title(f'Pawpularity: {int(label)}')

# Model defining and lightning functionality

- Model takes images and meta data as input
- using a pretrained cnn (we can finetune or freeze depending on situtation) and extract image features
- if network backbone is freezed we can pass images features to its trained classifier and get the class which network thinks present in image
- combine all features ie. image features + meta_features + classes(if using freezed backbone)
- pass these features to a regressor that gives pawpularity output

In [ ]:
class PawpularityModel(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super(PawpularityModel, self).__init__()
        self.cfg = cfg
        timm_model = timm.create_model(self.cfg.model_name, 
                                       pretrained=pretrained, 
                                       in_chans=3)
        
        if self.cfg.freeze_backbone:
            modules = []
            for module in timm_model.children():
                for param in module.parameters():
                    param.requires_grad = False
                modules.append(module)
        else:
            modules = list(timm_model.children())
        
        self.classifier = modules[-1]
        cnn_out_features = self.classifier.in_features
        classifier_out_features = self.classifier.out_features
        
        self.cnn = nn.Sequential(*modules[:-1])
        self.dropout = nn.Dropout(0.2)
        
        num_features = classifier_out_features + cfg.num_features + cnn_out_features
        self.regressor_1 = nn.Sequential(nn.Linear(num_features, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, cfg.targets))
        num_features = cnn_out_features + cfg.num_features
        self.regressor_2 = nn.Sequential(nn.Linear(num_features, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, cfg.targets))
        
    def forward(self, img, meta):
        cnn_features = self.cnn(img)
        if self.cfg.freeze_backbone:
            classes = self.classifier(cnn_features)
            cnn_features = self.dropout(cnn_features)
            x = torch.cat((cnn_features, meta, classes), dim=1)
            return self.regressor_1(x)
        cnn_features = self.dropout(cnn_features)
        x = torch.cat((cnn_features, meta), dim=1)
        return self.regressor_2(x)

### Lightning Module

In [ ]:
class LitPawpularity(pl.LightningModule):
    def __init__(self, cfg, model, fold):
        super(LitPawpularity, self).__init__()
        self.cfg = cfg
        self.model = model
        self.criterion = nn.MSELoss()
        self.rmse = metrics.MeanSquaredError(squared=False)
        self.r2 = metrics.R2Score()
        self.fold = fold
        
    def forward(self, img, meta):
        return self.model(img, meta)

    def configure_optimizers(self):
        self.optimizer = self.__get_optimizer()
        self.lr_scheduler = self.__get_lr_scheduler()
        return {'optimizer': self.optimizer, 'lr_scheduler': self.lr_scheduler}
    
    def training_step(self, batch, batch_idx):
        img, meta, y = batch
        y_hat = self(img, meta)
        loss = self.criterion(y_hat.view(-1), y)
        rmse = self.rmse(y_hat.view(-1), y)
        r2 = self.r2(y_hat.view(-1), y)
        logs = {'train_loss': loss, 
                'train_rmse': rmse,
                'train_r2': r2,
                'lr': self.optimizer.param_groups[0]['lr']}
        self.log_dict(
            logs,
            on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        img, meta, y = batch
        y_hat = self(img, meta)
        loss = self.criterion(y_hat.view(-1), y)
        rmse = self.rmse(y_hat.view(-1), y)
        r2 = self.r2(y_hat.view(-1), y)
        logs = {'val_loss': loss, 
                'val_rmse': rmse,
                'val_r2': r2}
        self.log_dict(
            logs,
            on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return logs
    
    def validation_epoch_end(self, outputs):
        rmse = torch.stack([x['val_rmse'] for x in outputs]).mean()
        loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        r2 = torch.stack([x['val_r2'] for x in outputs]).mean()
        print(f'Epoch {self.current_epoch} : Fold {self.fold} -> loss: {loss}\t rmse: {rmse}\t r2: {r2}')
        return outputs
    
    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        img, meta, _ = batch
        out = self(img, meta).flatten()
        return out

    def __get_optimizer(self):
        optimizer = None
        if self.cfg.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=self.cfg.lr, 
                                         weight_decay=self.cfg.weight_decay, 
                                         amsgrad=False)
        elif self.cfg.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.cfg.lr, 
                                          weight_decay=self.cfg.weight_decay)
        return optimizer

    def __get_lr_scheduler(self):
        scheduler = None
        if self.cfg.lr_scheduler=='ReduceLROnPlateau':
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', 
                                                                   factor=self.cfg.factor, 
                                                                   patience=self.cfg.patience, 
                                                                   eps=self.cfg.eps)
        elif self.cfg.lr_scheduler=='CosineAnnealingLR':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, 
                                                                   T_max=self.cfg.T_max, 
                                                                   eta_min=self.cfg.min_lr, 
                                                                   last_epoch=-1)
        elif self.cfg.lr_scheduler=='CosineAnnealingWarmRestarts':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optimizer,
                                                                             T_0=self.cfg.T_0,
                                                                             eta_min=self.cfg.min_lr,
                                                                             last_epoch=-1)
        return scheduler

### Lightning data module

In [ ]:
class PawpularityDataModule(pl.LightningDataModule):
    def __init__(self, 
                 train_files, 
                 val_files,
                 batch_size: int = 32,
                 image_size: int = Config.input_size,
                 apply_augmentations: bool = False):
        super(PawpularityDataModule, self).__init__()
        self.train_files = train_files
        self.val_files = val_files
        self.batch_size = batch_size
        self.augs = ImageAugmentations(image_size, apply_augs=apply_augmentations)

    def setup(self, stage=None):
        self.train_ds = PawpularityDataset(self.train_files,
                                           augmentations=self.augs.train_augs(),
                                           test=False)
        self.val_ds = PawpularityDataset(self.val_files,
                                         augmentations=self.augs.valid_augs(),
                                         test=False)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds,
                                           shuffle=True,
                                           num_workers=Config.num_workers, 
                                           pin_memory=True,
                                           drop_last=True,
                                           batch_size=self.batch_size)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_ds,
                                           shuffle=False,
                                           num_workers=Config.num_workers, 
                                           pin_memory=True,
                                           drop_last=False,
                                           batch_size=self.batch_size*2)

### Model checkpointing

In [ ]:
def model_checkpointing_callbacks(fold):
    return [pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                         dirpath="./",
                                         save_top_k=1, 
                                         save_last=False, 
                                         save_weights_only=True, 
                                         filename=f'best/{Config.model_name}_best_loss_fold{fold}', 
                                         verbose=True, 
                                         mode='min'),
            pl.callbacks.ModelCheckpoint(monitor='val_rmse',
                                         dirpath="./",
                                         save_top_k=1, 
                                         save_last=False, 
                                         save_weights_only=True, 
                                         filename=f'best/{Config.model_name}_best_rmse_fold{fold}', 
                                         verbose=True,
                                         mode='min')]

### Training Loggers

In [ ]:
def wandb_logger_init(name, job_type, **kwargs):
    return pl.loggers.WandbLogger(project=Config.project,
                                  name=name,
                                  group=Config.group, 
                                  job_type=job_type,
                                  config = {k:v for k,v in Config.__dict__.items() if not k.startswith("__")},
                                  **kwargs)

In [ ]:
def get_loggers(fold):
    if Config.loggers:
        return [wandb_logger_init(name=f"{Config.model_name}_fold{fold}", job_type="training"),
                pl.loggers.CSVLogger("logs", name=f"{Config.model_name}_fold{fold}")]
    return None

# Training

In [ ]:
if Config.train:
    cv = CrossValidation(train_csv, shuffle=True, random_state=Config.seed)
    for fold, (train_, val_) in enumerate(cv.kfold_split(splits=Config.n_fold)):
        if fold in Config.trn_folds:
            print(f"{'='*10} Fold {fold} {'='*10}")
            datamodule = PawpularityDataModule(train_, 
                                               val_,
                                               batch_size=Config.batch_size,
                                               apply_augmentations=True)
            datamodule.setup()
            loggers = get_loggers(fold)
            callbacks = [pl.callbacks.EarlyStopping(monitor='val_loss', 
                                                    patience=Config.patience, 
                                                    mode='min')]
            model = PawpularityModel(Config, pretrained=True)
            checkpoint_callbacks = model_checkpointing_callbacks(fold)
            lit = LitPawpularity(Config, model, fold=fold)
            trainer_params = {"max_epochs": Config.epochs,
                              "accumulate_grad_batches": Config.gradient_accumulation_steps,
                              "precision": Config.precision,
                              "callbacks": callbacks+checkpoint_callbacks,
                              "logger": loggers,
                              "progress_bar_refresh_rate": Config.print_freq}
            if Config.use_tpu:
                trainer_params["tpu_cores"] = 8
            else:
                trainer_params["gpus"] = -1
            trainer = pl.Trainer(**trainer_params)
            # Train the model
            trainer.fit(lit, datamodule)
            # Close wandb run
            wandb.finish()

# Saving OOF Predictions 

In [ ]:
def get_weights_path(fold, mode):
    return os.path.join("best", f"{Config.model_name}_best_{mode}_fold{fold}.ckpt")

In [ ]:
def get_fold_predictions(fold, data):
    print("="*10)
    print("Predictions using Fold: ", fold)
    print("="*10)
    augs = ImageAugmentations(Config.input_size, apply_augs=False)
    ds = PawpularityDataset(data,
                            augmentations=augs.valid_augs(),
                            test=False)
    weights = [get_weights_path(fold, "loss"),
               get_weights_path(fold, "rmse")]
    preds = []
    for weight in weights:
        print("Using weights: ", weight)
        loader = torch.utils.data.DataLoader(ds,
                                             shuffle=False,
                                             num_workers=Config.num_workers, 
                                             pin_memory=True,
                                             drop_last=False,
                                             batch_size=Config.batch_size*2)
        model = PawpularityModel(Config, pretrained=False)
        lit = LitPawpularity.load_from_checkpoint(weight, cfg=Config, model=model, fold=fold)
        trainer_params = {}
        if Config.use_tpu:
            trainer_params["tpu_cores"] = 8
        else:
            trainer_params["gpus"] = -1
        trainer = pl.Trainer(**trainer_params)
        predictions = trainer.predict(lit, loader)
        predictions = torch.cat([x for x in predictions]).detach().cpu().numpy()
        preds.append(predictions)
    preds = np.mean(np.column_stack(preds), axis=1)
    return preds

In [ ]:
oof_df = pd.DataFrame()
cv = CrossValidation(train_csv, shuffle=True, random_state=Config.seed)
for fold, (_, val_) in enumerate(cv.kfold_split(splits=Config.n_fold)):
    if fold in Config.trn_folds:
        _oof_df = pd.DataFrame()
        _oof_df["Id"] = val_["Id"]
        _oof_df["Pawpularity"] = val_["Pawpularity"]
        _oof_df["Predictions"] = get_fold_predictions(fold, val_)
        oof_df = pd.concat([oof_df, _oof_df])
oof_df.to_csv(f"oof_{Config.model_name}.csv", index=False)

In [ ]:
oof = pd.read_csv(f"oof_{Config.model_name}.csv")
oof.head()

# 🐕🐕🐕🐕🐕🦮🦮🦮🦮🦮🐄🐄🐄🐄🐄🐄